# LangGraph入門

## 1.準備

In [ ]:
# 必要なモジュールをインポート
from dotenv import load_dotenv
from IPython.display import Image, display

# 環境変数の読み込み
load_dotenv()

# モデル名
MODEL_NAME = "gpt-5-mini"

## 2.単純なグラフ


In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Any

# 1. Stateの定義
class State(TypedDict):
    value: int

# 2. ノード関数の定義
def node_add1(state: State) -> dict[str, Any]:
    return {"value": state["value"] + 1}

def node_double(state: State) -> dict[str, Any]:
    return {"value": state["value"] * 2}

# 3. グラフの構築
graph = StateGraph(State)

graph.add_node("add", node_add1)
graph.add_node("double", node_double)

# STARTノードからaddへ接続
graph.add_edge(START, "add")
graph.add_edge("add", "double")
graph.add_edge("double", END)

# コンパイル
app = graph.compile()

In [ ]:
# グラフの可視化
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# 初期値として value に 10 を渡す
initial_input = {"value": 10}

# グラフを実行 (invoke)
result = app.invoke(initial_input)

print(f"実行結果: {result}")

In [ ]:
print("--- 実行プロセスの追跡 ---")
inputs = {"value": 10}

# 各ステップごとに何が起きたかを出力
for step in app.stream(inputs):
    print(step)

## 3.状態の保持、条件分岐

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from langgraph.types import Command

# Stateの定義
class State(TypedDict):
    count: int
    value: int

# カウンタを1アップし、条件によって遷移先を変えるノード
def node_count(state: State) -> Command[Literal["node_double", END]]:
    new_count = state["count"] + 1

    # 状態更新と同時に、次の遷移先を指定（Command）
    if new_count < 3:
        print(f"Count: {new_count} -> Next: double")
        return Command(
            update={"count": new_count},
            goto="node_double"
        )
    else:
        print(f"Count: {new_count} -> Finish")
        return Command(
            update={"count": new_count},
            goto=END
        )

def node_double(state: State) -> Command[Literal["node_count"]]:
    # 単純な遷移もCommandで記述可能（gotoのみ指定も可）
    return Command(
        update={"value": state["value"] * 2},
        goto="node_count"
    )

In [ ]:
# グラフの構築
graph = StateGraph(State)
graph.add_node("node_count", node_count)
graph.add_node("node_double", node_double)

# エッジ定義は「開始地点からnode_count」だけでOK
graph.add_edge(START, "node_count") 

app = graph.compile()

In [ ]:
# グラフの可視化
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
print("--- 実行プロセスの追跡 ---")
inputs = {"count": 0, "value": 1}

# 各ステップごとに何が起きたかを出力
for step in app.stream(inputs):
    print(step)

## 4.LLMの呼出


In [ ]:
from typing import TypedDict, Annotated, Any
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 1. ステートの定義
class State(TypedDict):
    # add_messages はメッセージの追加・更新（IDベース）を自動処理するReducer
    messages: Annotated[list[BaseMessage], add_messages]
    animal: str
    voice: str

In [ ]:
# 2. プロンプトとモデルの準備
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは{animal}らしく、語尾に{voice}などと付けて答えます。"),
    MessagesPlaceholder(variable_name="messages"),
])

model = ChatOpenAI(model=MODEL_NAME)

# チェーンの作成
my_chain = prompt | model

# 3. ノードの定義
def chatbot(state: State) -> dict[str, Any]:
    # チェーンの実行
    response = my_chain.invoke({
        "animal": state["animal"],
        "voice": state["voice"],
        "messages": state["messages"],
    })

    # 更新差分を返す（add_messagesにより、既存リストに追加される）
    return {"messages": [response]}

In [ ]:
# 4. グラフの構築
graph = StateGraph(State)

# ノードの追加
graph.add_node("chatbot", chatbot)

# エッジの定義
graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

# グラフのコンパイル
app = graph.compile()

# 5. グラフの可視化
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# 6. グラフの実行
input_data = {
    "animal": "犬",
    "voice": "ワン！",
    "messages": [HumanMessage(content="英語学習をする上でのポイントは？")]
}

# 最終結果の表示
response = app.invoke(input_data)
print(response["messages"][-1].content)

# 5.任意の関数をノードとして定義する

In [ ]:
from typing import TypedDict, Any
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from IPython.display import Image, display

# 1. ステートの定義
class State(TypedDict):
    question: str       # ユーザーからの質問
    raw_response: str   # LLMが生成した生のテキスト
    final_answer: str   # 加工後の最終回答

In [ ]:
# モデルの準備
model = ChatOpenAI(model=MODEL_NAME)

# 2. ノードの定義
# プロンプトの準備
prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}\n\n上記の質問について簡潔に回答してください。"),
])

# チェーンの作成と実行
my_chain = prompt | model | StrOutputParser()    

# ノードA: LLMによる回答生成のみを担当
def generation_node(state: State) -> dict[str, Any]:
    response = my_chain.invoke({"question": state["question"]})
    
    # 状態の「差分」を返す。これにより state["raw_response"] が更新される
    return {"raw_response": response}

# ノードB: 任意の関数（文字列操作）を担当
def reverse_string_node(state: State) -> dict[str, Any]:
    # 前のノードの結果をステートから取得
    original_text = state["raw_response"]
    
    # 任意の関数処理（ここでは文字列反転）
    reversed_text = original_text[::-1]
    
    # 処理結果を最終回答としてステートに書き込む
    return {"final_answer": reversed_text}

In [ ]:
# 3. グラフの構築
builder = StateGraph(State)

# ノードを追加
builder.add_node("generate", generation_node)
builder.add_node("reverse_process", reverse_string_node)

# エッジ（流れ）の定義
# START -> 生成 -> 加工 -> END
builder.add_edge(START, "generate")
builder.add_edge("generate", "reverse_process")
builder.add_edge("reverse_process", END)

# コンパイル
app = builder.compile()

# グラフの可視化
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# グラフの実行
inputs = {"question": "あなたの趣味は何ですか？"}
result = app.invoke(inputs)

print(f"質問: {result['question']}")
print(f"LLMの回答 (raw): {result['raw_response']}")
print(f"関数処理後 (final): {result['final_answer']}")

## 6.複数のノードを並列につなげる


In [ ]:
from typing import TypedDict, Any
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# 1. ステートの定義
# TypedDictを使用して型を定義します。
# 並列実行でも、キーが異なれば競合せずにマージされます。
class State(TypedDict):
    question: str
    positive: str
    negative: str
    answer: str

In [ ]:
# 2. プロンプトとチェーンの準備
positive_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは楽観主義者です。ユーザーからの質問に対して常に前向きな回答をします。"),
    ("human", "{question}"),
])

negative_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは悲観主義者です。ユーザーからの質問に対して常に否定的な回答をします。"),
    ("human", "{question}"),
])

opinion_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは平等主義者です。2つの意見を平等にまとめます。まとめた結果だけを出力します。"),
    ("human", "楽観的な意見: {positive}\n悲観的な意見: {negative}"),
])

# モデルのセットアップ
model = ChatOpenAI(model=MODEL_NAME)

# 3. ノードの定義
positive_chain = positive_prompt | model | StrOutputParser()
def positive_node(state: State) -> dict[str, Any]:
    return {"positive": positive_chain.invoke({"question": state["question"]})}

negative_chain = negative_prompt | model | StrOutputParser()
def negative_node(state: State) -> dict[str, Any]:
    return {"negative": negative_chain.invoke({"question": state["question"]})}

opinion_chain = opinion_prompt | model | StrOutputParser()
def opinion_node(state: State) -> dict[str, Any]:
    # ここに来る時点で、positiveとnegativeの両方がstateに入っています
    return {"answer": opinion_chain.invoke(state)}

In [ ]:
# 4. グラフの構築
builder = StateGraph(State)

# ノードの追加
builder.add_node("positive_agent", positive_node)
builder.add_node("negative_agent", negative_node)
builder.add_node("opinion_agent", opinion_node)

# 並列実行の定義 (Fork)
# STARTから複数のノードにエッジを張ることで、これらが同時に起動します
builder.add_edge(START, "positive_agent")
builder.add_edge(START, "negative_agent")

# 合流の定義 (Join)
# 両方のエッジを同じノードに向けることで、両方の処理が終わった後に次が実行されます
# LangGraphのスーパー・ステップ機能により、自動的に待機・マージが行われます
builder.add_edge("positive_agent", "opinion_agent")
builder.add_edge("negative_agent", "opinion_agent")

# 終了定義
builder.add_edge("opinion_agent", END)

# コンパイル
app = builder.compile()

# 5. グラフの可視化と実行
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# 6. グラフの実行
print("--- 実行開始 ---")
initial_state = {"question": "AIの進化が人間に与える影響は？"}

# streamで実行経過を確認
for output in app.stream(initial_state):
    for key, value in output.items():
        print(f"\nCompleted Node: {key}")
        # 出力が見やすいように整形
        if "positive" in value:
            print(f"Positive: {value['positive'][:50]}...")
        elif "negative" in value:
            print(f"Negative: {value['negative'][:50]}...")
        elif "answer" in value:
            print(f"\nFinal Answer:\n{value['answer']}")

## 7.チャットボットの作成

In [ ]:
from typing import TypedDict, Annotated, Any
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from IPython.display import Image, display

# 1. ステートの定義
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# 2. プロンプトとモデルの準備
prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは親切なアシスタントです。"),
    MessagesPlaceholder(variable_name="messages")
])

model = ChatOpenAI(model=MODEL_NAME)

# チェーンの構築
my_chain = prompt | model

# 3. ノードの定義
def chatbot_node(state: State) -> dict[str, Any]:
    response = my_chain.invoke(state)
    
    # 更新差分（AIのメッセージ）を返します
    return {"messages": [response]}

In [ ]:
import uuid # UUID（ランダムなID）生成用
from langgraph.checkpoint.memory import InMemorySaver # メモリ保存用

# 4. グラフの構築
builder = StateGraph(State)

builder.add_node("chatbot", chatbot_node)

# エッジの定義 (START -> chatbot -> END)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

# Checkpointerの初期化
memory = InMemorySaver()

# コンパイル時にcheckpointerを渡す
# これにより、グラフはスレッドIDに基づいて状態を保存・復元できるようになる
agent = builder.compile(checkpointer=memory)

# グラフの可視化
display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
# ランダムなUUIDを生成して thread_id に設定
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

while True:
    # ユーザーからの質問を受付
    user_input = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if user_input.strip() == "":
        break
    print(f"質問：{user_input}")
    
    # エージェントを実行し、応答をストリーミング表示
    # configを渡すことで、メモリから過去の会話をロードします
    for chunk, metadata in agent.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config, 
        stream_mode="messages"
    ):
        # chunkは AIMessageChunk オブジェクトなどが返ってきます
        if hasattr(chunk, "content") and chunk.content:
            print(chunk.content, end="", flush=True)
            
    print() # 改行

print("\n---ご利用ありがとうございました！---")